In [2]:
import pandas as pd
import numpy as np
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules



In [3]:
trainDf = pd.read_csv("C:/Users/shiv/MARKET BASKET ANALYSIS/Data/order_products__train.csv")
orderDf = pd.read_csv("C:/Users/shiv/MARKET BASKET ANALYSIS/Data/orders.csv")
productDf = pd.read_csv("C:/Users/shiv/MARKET BASKET ANALYSIS/Data/products.csv")

In [4]:
priorDf= pd.read_csv("C:/Users/shiv/MARKET BASKET ANALYSIS/Data/order_products__prior.csv")


In [5]:
trainDf.head()

,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1


In [6]:
trainDf.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99999 entries, 0 to 99998
Data columns (total 4 columns):
order_id             99999 non-null int64
product_id           99999 non-null int64
add_to_cart_order    99999 non-null int64
reordered            99999 non-null int64
dtypes: int64(4)
memory usage: 3.1 MB


In [7]:
converted_trainDf = trainDf.astype(np.uint8)

In [8]:
converted_trainDf.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99999 entries, 0 to 99998
Data columns (total 4 columns):
order_id             99999 non-null uint8
product_id           99999 non-null uint8
add_to_cart_order    99999 non-null uint8
reordered            99999 non-null uint8
dtypes: uint8(4)
memory usage: 390.7 KB


In [9]:
trainDf = trainDf.append(priorDf,ignore_index = True)

In [10]:
trainDf['reordered'] = 1 

In [11]:
productCountDf = trainDf.groupby("product_id",as_index = False)["order_id"].count()

In [12]:
productCountDf.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21927 entries, 0 to 21926
Data columns (total 2 columns):
product_id    21927 non-null int64
order_id      21927 non-null int64
dtypes: int64(2)
memory usage: 513.9 KB


In [13]:
topLev = 100

#Here order_id is the count so we need to sort the data frame w.r.t order_id
productCountDf = productCountDf.sort_values("order_id",ascending = False)

topProdFrame = productCountDf.iloc[0:topLev,:]
topProdFrame = topProdFrame.merge(productDf,on = "product_id")
productId= topProdFrame.loc[:,["product_id"]]

In [14]:
df = trainDf[0:0]
for i in range(0,99):
    pId = productId.iloc[i]['product_id'] 
    stDf = trainDf[trainDf.product_id == pId ]
    df = df.append(stDf,ignore_index = False)

In [15]:
df.head()

,order_id,product_id,add_to_cart_order,reordered
115,226,24852,2,1
156,473,24852,2,1
196,878,24852,2,1
272,1042,24852,1,1
297,1139,24852,1,1


In [16]:
df.info(memory_usage='deep')


<class 'pandas.core.frame.DataFrame'>
Int64Index: 45431 entries, 115 to 199399
Data columns (total 4 columns):
order_id             45431 non-null int64
product_id           45431 non-null int64
add_to_cart_order    45431 non-null int64
reordered            45431 non-null int64
dtypes: int64(4)
memory usage: 1.7 MB


In [17]:
converted_df=df.astype('float16')

In [18]:
converted_df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45431 entries, 115 to 199399
Data columns (total 4 columns):
order_id             45431 non-null float16
product_id           45431 non-null float16
add_to_cart_order    45431 non-null float16
reordered            45431 non-null float16
dtypes: float16(4)
memory usage: 709.9 KB


In [19]:
basket = df.groupby(['order_id', 'product_id'])['reordered'].sum().unstack().reset_index().fillna(0).set_index('order_id')

In [20]:
basket.head()

product_id,196,3957,4210,4605,4799,4920,5077,5450,5785,5876,...,45066,46667,46979,47144,47209,47626,47766,48679,49235,49683
order_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
basket.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14091 entries, 1 to 246834
Data columns (total 99 columns):
196      14091 non-null float64
3957     14091 non-null float64
4210     14091 non-null float64
4605     14091 non-null float64
4799     14091 non-null float64
4920     14091 non-null float64
5077     14091 non-null float64
5450     14091 non-null float64
5785     14091 non-null float64
5876     14091 non-null float64
8174     14091 non-null float64
8277     14091 non-null float64
8424     14091 non-null float64
8518     14091 non-null float64
9076     14091 non-null float64
9387     14091 non-null float64
9839     14091 non-null float64
10749    14091 non-null float64
11520    14091 non-null float64
12341    14091 non-null float64
13176    14091 non-null float64
15290    14091 non-null float64
16759    14091 non-null float64
16797    14091 non-null float64
17794    14091 non-null float64
18465    14091 non-null float64
19057    14091 non-null float64
19660    14091 non-null fl

In [22]:
converted_basket=basket.astype('float16')

In [23]:
converted_basket.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14091 entries, 1 to 246834
Data columns (total 99 columns):
196      14091 non-null float16
3957     14091 non-null float16
4210     14091 non-null float16
4605     14091 non-null float16
4799     14091 non-null float16
4920     14091 non-null float16
5077     14091 non-null float16
5450     14091 non-null float16
5785     14091 non-null float16
5876     14091 non-null float16
8174     14091 non-null float16
8277     14091 non-null float16
8424     14091 non-null float16
8518     14091 non-null float16
9076     14091 non-null float16
9387     14091 non-null float16
9839     14091 non-null float16
10749    14091 non-null float16
11520    14091 non-null float16
12341    14091 non-null float16
13176    14091 non-null float16
15290    14091 non-null float16
16759    14091 non-null float16
16797    14091 non-null float16
17794    14091 non-null float16
18465    14091 non-null float16
19057    14091 non-null float16
19660    14091 non-null fl

In [24]:
# Convert the units to 1 hot encoded values
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

In [25]:
basket_sets = converted_basket.applymap(encode_units)

In [26]:
basket_sets.head()

product_id,196,3957,4210,4605,4799,4920,5077,5450,5785,5876,...,45066,46667,46979,47144,47209,47626,47766,48679,49235,49683
order_id,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
basket_sets.size

1395009

In [28]:
basket_sets.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14091 entries, 1 to 246834
Data columns (total 99 columns):
196      14091 non-null int64
3957     14091 non-null int64
4210     14091 non-null int64
4605     14091 non-null int64
4799     14091 non-null int64
4920     14091 non-null int64
5077     14091 non-null int64
5450     14091 non-null int64
5785     14091 non-null int64
5876     14091 non-null int64
8174     14091 non-null int64
8277     14091 non-null int64
8424     14091 non-null int64
8518     14091 non-null int64
9076     14091 non-null int64
9387     14091 non-null int64
9839     14091 non-null int64
10749    14091 non-null int64
11520    14091 non-null int64
12341    14091 non-null int64
13176    14091 non-null int64
15290    14091 non-null int64
16759    14091 non-null int64
16797    14091 non-null int64
17794    14091 non-null int64
18465    14091 non-null int64
19057    14091 non-null int64
19660    14091 non-null int64
20114    14091 non-null int64
20995    14091 non-n

In [29]:
np.iinfo('uint8')

iinfo(min=0, max=255, dtype=uint8)

In [30]:
converted_basket_sets = basket_sets.astype(np.uint8)

In [31]:
converted_basket_sets.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14091 entries, 1 to 246834
Data columns (total 99 columns):
196      14091 non-null uint8
3957     14091 non-null uint8
4210     14091 non-null uint8
4605     14091 non-null uint8
4799     14091 non-null uint8
4920     14091 non-null uint8
5077     14091 non-null uint8
5450     14091 non-null uint8
5785     14091 non-null uint8
5876     14091 non-null uint8
8174     14091 non-null uint8
8277     14091 non-null uint8
8424     14091 non-null uint8
8518     14091 non-null uint8
9076     14091 non-null uint8
9387     14091 non-null uint8
9839     14091 non-null uint8
10749    14091 non-null uint8
11520    14091 non-null uint8
12341    14091 non-null uint8
13176    14091 non-null uint8
15290    14091 non-null uint8
16759    14091 non-null uint8
16797    14091 non-null uint8
17794    14091 non-null uint8
18465    14091 non-null uint8
19057    14091 non-null uint8
19660    14091 non-null uint8
20114    14091 non-null uint8
20995    14091 non-n

In [38]:
frequent_itemsets = apriori(converted_basket_sets, min_support=0.01, use_colnames=True)

In [39]:
frequent_itemsets

,support,itemsets
0,0.016535,(196)
1,0.014193,(3957)
2,0.014619,(4210)
3,0.032361,(4605)
4,0.015258,(4799)
...,...,...
133,0.011284,"(49683, 24852)"
134,0.013271,"(26209, 47626)"
135,0.011071,"(26209, 47766)"
136,0.010290,"(47209, 27966)"


In [43]:
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=2)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(13176),(27966),0.165780,0.057980,0.019445,0.117295,2.023007,0.009833,1.067196
1,(27966),(13176),0.057980,0.165780,0.019445,0.335373,2.023007,0.009833,1.255171
2,(21137),(27966),0.110993,0.057980,0.014619,0.131714,2.271696,0.008184,1.084918
3,(27966),(21137),0.057980,0.110993,0.014619,0.252142,2.271696,0.008184,1.188738
4,(21137),(39275),0.110993,0.043574,0.010929,0.098465,2.259734,0.006093,1.060887
5,(39275),(21137),0.043574,0.110993,0.010929,0.250814,2.259734,0.006093,1.186631
6,(26209),(47626),0.059329,0.074232,0.013271,0.223684,3.013321,0.008867,1.192515
7,(47626),(26209),0.074232,0.059329,0.013271,0.178776,3.013321,0.008867,1.145451
8,(26209),(47766),0.059329,0.079767,0.011071,0.186603,2.339343,0.006338,1.131345
9,(47766),(26209),0.079767,0.059329,0.011071,0.138790,2.339343,0.006338,1.092267


In [44]:
rules[ (rules['lift'] >= 2) &
       (rules['confidence'] >= 0.1) ]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(13176),(27966),0.165780,0.057980,0.019445,0.117295,2.023007,0.009833,1.067196
1,(27966),(13176),0.057980,0.165780,0.019445,0.335373,2.023007,0.009833,1.255171
2,(21137),(27966),0.110993,0.057980,0.014619,0.131714,2.271696,0.008184,1.084918
3,(27966),(21137),0.057980,0.110993,0.014619,0.252142,2.271696,0.008184,1.188738
5,(39275),(21137),0.043574,0.110993,0.010929,0.250814,2.259734,0.006093,1.186631
6,(26209),(47626),0.059329,0.074232,0.013271,0.223684,3.013321,0.008867,1.192515
7,(47626),(26209),0.074232,0.059329,0.013271,0.178776,3.013321,0.008867,1.145451
8,(26209),(47766),0.059329,0.079767,0.011071,0.186603,2.339343,0.006338,1.131345
9,(47766),(26209),0.079767,0.059329,0.011071,0.138790,2.339343,0.006338,1.092267
10,(47209),(27966),0.081470,0.057980,0.010290,0.126307,2.178441,0.005567,1.078204
